<a id='content'></a>
# Содержание
<br><b>1. [Выгрузка и обзор данных](#first)</b>
<br><b>2. [Выделение обучающей, валиданционной, тестовой выборок](#second)</b>
<br><b>3. [Дерево решений](#third)</b>
<br><b>4. [Случайный лес](#forth)</b>
<br><b>5. [Логистическая регрессия](#fivth)</b>
<br><b>6. [Проверки на тестовой выборке](#sixth)</b>

На основании предоставленных данных необходимо определить наилучшую модель, позволяющую рекомендовать абонентам отпимальный тарифный план, а также подобрать гиперпараметры.

<a id='first'></a>
# 1. Выгрузка и обзор днных.
[содержание](#content) [>>](#second)

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# df = pd.read_csv('users_behavior.csv')
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Данные корректны. Пропусков нет. Можно преступать к выделению выборок.

<a id='second'></a>
# 2. Выделение обучающей, валиданционной, тестовой выборок
[<<](#first) [содержание](#content) [>>](#third)

Выделим целевой показатель в отдельную таблицу.

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Выделим на тестовую и валидационную выборки по 20% от общих данных.

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, random_state=8323, test_size=.2)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, random_state=8323, test_size=.25)

In [8]:
features_train

,calls,minutes,messages,mb_used
3072,74.0,561.64,27.0,23297.82
618,65.0,490.26,76.0,30307.39
1202,49.0,316.69,46.0,17519.29
2144,44.0,324.86,0.0,18611.43
1261,80.0,552.33,23.0,18457.85
...,...,...,...,...
2727,34.0,210.03,49.0,17071.06
2564,58.0,444.38,42.0,19477.10
2615,51.0,296.04,38.0,22479.57
1833,31.0,197.82,13.0,2237.30


In [9]:
target_train.head()

3072    0
618     1
1202    0
2144    0
1261    0
Name: is_ultra, dtype: int64

<a id='third'></a>
# 3. Дерево решений
[<<](#second) [содержание](#content) [>>](#forth)

Для получения максимально точно предсказывающего алгоритма проведем обучение на деревьях с глубиной от 1 до 10.

In [11]:
for depth in range(1, 11):
    dec_tree_model = DecisionTreeClassifier(random_state=8323, max_depth=depth)
    dec_tree_model.fit(features_train, target_train)
    dec_tree_prediction = dec_tree_model.predict(features_valid)
    dec_tree_accuracy = accuracy_score(target_valid, dec_tree_prediction)
    print('max_depth =', depth, ':', dec_tree_accuracy)

max_depth = 1 : 0.7527216174183515
max_depth = 2 : 0.7744945567651633
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7916018662519441
max_depth = 6 : 0.7947122861586314
max_depth = 7 : 0.7962674961119751
max_depth = 8 : 0.7962674961119751
max_depth = 9 : 0.8009331259720062
max_depth = 10 : 0.7900466562986003


Лучшие показатели получены со значением глубины дерева 9. Начиная с глубины 5 значения accuracy улучшается в пределах 1%. Далее начинает снижаться.

<a id='forth'></a>
# 4. Случайный лес
[<<](#third) [содержание](#content) [>>](#fivth)

Будем менять число деревьев от 1 до 50 с шагом 10. При каждом из значений будем так же менять и глубину деревьев от 1 до 10. Так определим гиперпараметры обеспечившие максимальную точность.

In [13]:
best_model = None
best_depth = 0
best_est = 0
best_accuracy = 0

for est in range(1, 51, 10):
    for depth in range(1, 10):
        rnd_forest_model = RandomForestClassifier(random_state=8323, max_depth=depth, n_estimators=est)
        rnd_forest_model.fit(features_train, target_train)
        rnd_forest_prediction = rnd_forest_model.predict(features_valid)
        rnd_forest_accuracy = accuracy_score(target_valid, rnd_forest_prediction)
        if rnd_forest_accuracy > best_accuracy:
            best_accuracy = rnd_forest_accuracy
            best_est = est
            best_depth = depth
            best_model = rnd_forest_model

print('Максимальная точность:', best_accuracy,'\n', 
      'Глубина при максимальной точности:', best_depth, '\n',
     'estimators прим максимальной точности:', best_est)

Максимальная точность: 0.8180404354587869 
 Глубина при максимальной точности: 9 
 estimators прим максимальной точности: 11


Точность на 1,8% выше чем у решающего дерева.

<a id='fivth'></a>
# 5. Логистическая регрессия
[<<](#forth) [содержание](#content) [>>](#sixth)

In [15]:
lr_model = LogisticRegression(random_state=8323, max_iter=1000, solver='lbfgs')
lr_model.fit(features_train, target_train)
# lr_prediction = lr_model.predict(features_valid)
lr_score = lr_model.score(features_valid, target_valid)
print(lr_score)

0.7122861586314152


Самый низки показатель точности.

<div class="alert alert-block alert-warning">
привести признаки к одному масштабу. Если же хочешь использовать классический вариант логистической регрессии стоит использовать penalty='none'
</div>

<a id='sixth'></a>
# 6. Проверки на тестовой выборке
[<<](#fivth) [содержание](#content) [>>](#seventh)

## 6.1. Решающее дерево

Выбираем глубину дерева 9, показавшую максимальную точность на валидационной выборке.

In [16]:
dec_tree_model_test = DecisionTreeClassifier(random_state=8323, max_depth=9)
dec_tree_model_test.fit(features_train, target_train)
dec_tree_prediction_test = dec_tree_model_test.predict(features_test)
dec_tree_accuracy_test = accuracy_score(target_test, dec_tree_prediction_test)
print(dec_tree_accuracy_test)

0.7853810264385692


## 6.2. Случайный лес

In [17]:
rnd_forest_model_test = RandomForestClassifier(random_state=8323, max_depth=9, n_estimators=11)
rnd_forest_model_test.fit(features_train, target_train)
rnd_forest_prediction_test = rnd_forest_model_test.predict(features_test)
accuracy_score(target_test, rnd_forest_prediction_test)

0.807153965785381

## 6.3. Логистическая регрессия

Так как мы не меняли гиперпараметры у модели логистической регрессии, то нет необходимости переобучать модель. Рассчитаем точность предсказаний тестовой выборке.

In [18]:
lr_score = lr_model.score(features_test, target_test)
print(lr_score)

0.6951788491446346


<a id='conclusion'></a>
# Вывод
[<<](#sixth) [содержание](#content)

<li>Максимальную точно предсказаний показала модель Случайный лес. Далее идет решающее дерево. Самый низкий показатель у Логистической регрессии.